# 02 Working with EMIT L2A Reflectance and ECOSTRESS L2 LSTE ET Products

---

**Summary**  

In the previous notebook, we found and downloaded concurrent EMIT L2A Reflectance and ECOSTRESS L2 Land Surface Temperature and Emissivity scenes over our region of interest. In this notebook, we will open and explore both datasets to better understand the structure, then we will conduct some common preprocessing routines to make the data usable together, including: applying quality data, reprojecting, placing data on a common grid, and cropping. 

**Background**

**References**
- Leith, Alex. 2023. Hyperspectral Notebooks. Jupyter Notebook. Auspatious. https://github.com/auspatious/hyperspectral-notebooks/tree/main

#TODO - Add some text about ECOSTRESS and EMIT Synergies and general info about reflectance and lst

**Requirements** 
 - [NASA Earthdata Account](https://urs.earthdata.nasa.gov/home) 
 - *No Python setup requirements if connected to the workshop cloud instance!*
 - Set up Python Environment - See **setup_instructions.md** in the `/setup/` folder

**Learning Objectives**  
- How to open and work with EMIT L2A Reflectance and ECOSTRESS L2T LSTE data
- How to apply quality data to EMIT and ECOSTRESS scenes
- How to reproject and regrid data
- How to crop EMIT and ECOSTRESS data

**Tutorial Outline**  

1. Setup  
2. Opening and Exploring EMIT Data  
    2.1 Applying Quality Masks to EMIT Data  
    2.2 Cropping EMIT Data  
    2.3 Writing Outputs  
3. Opening and Exploring ECOSTRESS Data  
    3.1 Applying Quality Masks to ECOSTRESS Data  
    3.2 Reprojecting and Regridding ECOSTRESS Data  
    3.3 Cropping ECOSTRESS Data  
    3.4 Writing Outputs  

## 1 Setup 

---

### 1.1 Import Python Libraries



In [ ]:
# Import Packages
import os
import glob
import math
import earthaccess
import numpy as np
import pandas as pd
import xarray as xr
from osgeo import gdal
import rasterio as rio
import rioxarray as rxr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import geoviews as gv
import geopandas as gp
import sys
from modules.emit_tools import emit_xarray, ortho_xr
#from modules.download_granules import main as dl_gran
#from holoviews import opts
from holoviews.plotting.links import DataLink

### 1.2 Define Filepaths for one type of each file downloaded

Define a filepath for an EMIT L2A Reflectance file, EMIT L2A Mask file, and an ECOSTRESS L2T LSTE and ECOSTRESS L2T Mask file.

In [ ]:
emit_fp = "../../shared/2023-VITALS-Workshop-AGU/data/EMIT_L2A_RFL_001_20230401T203751_2309114_002.nc"
emit_qa_fp = "../../shared/2023-VITALS-Workshop-AGU/data/EMIT_L2A_MASK_001_20230401T203751_2309114_002.nc"
eco_fp = "../../shared/2023-VITALS-Workshop-AGU/data/ECOv002_L2T_LSTE_26860_001_10SGD_20230401T203733_0710_01_LST.tif"

## 2. Opening and Exploring EMIT Reflectance Data

EMIT L2A Reflectance Data are distributed in a non-orthocorrected spatially raw NetCDF4 (.nc) format consisting of the data and its associated metadata. Inside the L2A Reflectance `.nc` file there are 3 groups. Groups can be thought of as containers to organize the data. 

1. The root group that can be considered the main dataset contains the reflectance data described by the downtrack, crosstrack, and bands dimensions.  
2. The `sensor_band_parameters`  group containing the wavelength center and the full-width half maximum (FWHM) of each band.  
3. The `location` group contains latitude and longitude values at the center of each pixel described by the crosstrack and downtrack dimensions, as well as a geometry lookup table (GLT) described by the ortho_x and ortho_y dimensions. The GLT is an orthorectified image (EPSG:4326) consisting of 2 layers containing downtrack and crosstrack indices. These index positions allow us to quickly project the raw data onto this geographic grid.

To work with the EMIT data, we will use the `emit_tools` module. There are other ways to work with the data and a more thorough explanation of the `emit_tools` in the [EMIT-Data-Resources Repository](https://github.com/nasa/EMIT-Data-Resources).

Open the example EMIT scene using the `emit_xarray` function. In this step we will use the `ortho=True` argument to orthorectify the scene using the included GLT. 

In [ ]:
emit_ds = emit_xarray(emit_fp, ortho=True)
emit_ds

We can plot the spectra of an individual pixel closest to a latitude and longitude we want using the `sel` function from `xarray`. Using the `good_wavelengths` flag from the `sensor_band_parameters` group, mask out bands where water absorption features were assigned a value of -0.01 reflectance. Typically data around 1320-1440 nm and 1770-1970 nm is noisy due to the moisture present in the atmosphere; therefore, these spectral regions offer little information about targets and can be excluded from calculations. 

In [ ]:
emit_ds = emit_ds.fillna(np.nan).where(emit_ds.reflectance!=-0.01)

Now select a point and plot a spectra. In this example, we'll first find the center of the scene and use those coordinates.

In [ ]:
scene_center = emit_ds.latitude.values[int(len(emit_ds.latitude)/2)],emit_ds.longitude.values[int(len(emit_ds.longitude)/2)]
scene_center

In [ ]:
point = emit_ds.sel(latitude=scene_center[0],longitude=scene_center[1], method='nearest')
point.hvplot.line(y='reflectance',x='wavelengths', color='black').opts(
    title=f'Latitude = {point.latitude.values.round(3)}, Longitude = {point.longitude.values.round(3)}')

We can also plot individual bands spatially by selecting a wavelength, then plotting. Select the band with a wavelengths of 850 nm and plot it using ESRI imagery as a basemap to get a better understanding of where the scene was acquired. 

In [ ]:
emit_layer = emit_ds.sel(wavelengths=850,method='nearest')
emit_layer.hvplot.image(cmap='viridis',geo=True, tiles='ESRI', frame_width=720,frame_height=405, alpha=0.7, fontscale=2).opts(
    title=f"{emit_layer.wavelengths:.3f} {emit_layer.wavelengths.units}", xlabel='Longitude',ylabel='Latitude')

### Interactive Spectral Plots

Combining the Spatial and Spectral information into a single visualization can be a powerful tool for exploring and inspecting imaging spectroscopy data. Using the streams module from Holoviews we can link a spatial map to a plot of spectra.

We could plot a single band image as we previously have, but using a multiband image, like an RGB may help infer what targets we're examining. Build an RGB image following the steps below.

Select bands to represent red (650 nm), green (560 nm), and blue (470 nm) by finding the nearest to a wavelength chosen to represent that color.

In [ ]:
emit_rgb = emit_ds.sel(wavelengths=[650, 560, 470], method='nearest')

We may need to adjust balance the brightness of the selected wavelengths to make a prettier map. **This will not affect the data, just the visuals.** To do this we will use the function below. We can change the `bright` argument to increase or decrease the brightness of the scene as a whole. A value of 0.2 usually works pretty well.

In [ ]:
def gamma_adjust(rgb_ds, bright=0.2, white_background=False):
    array = rgb_ds.reflectance.data
    gamma = math.log(bright)/math.log(np.nanmean(array)) # Create exponent for gamma scaling - can be adjusted by changing 0.2 
    scaled = np.power(array,gamma).clip(0,1) # Apply scaling and clip to 0-1 range
    if white_background == True:
        scaled = np.nan_to_num(scaled, nan = 1) # Assign NA's to 1 so they appear white in plots
    rgb_ds.reflectance.data = scaled
    return rgb_ds

#TODO - Investigate invalid values here - probably nans

In [ ]:
emit_rgb = gamma_adjust(emit_rgb,white_background=True)

Now that we have an RGB dataset, we can use that to create a spatial plot, and data selected by clicking on that 'map' can be inputs for a function to return values from the full dataset at that latitude and longitude location using the cell below.

In [ ]:
#TODO - This still needs some work, sizing, normalizing scene brightness, fixing competing toolbar values - double import?

In [ ]:
# Interactive Points Plotting
# Modfied from https://github.com/auspatious/hyperspectral-notebooks/blob/main/03_EMIT_Interactive_Points.ipynb
POINT_LIMIT = 20
color_cycle = hv.Cycle('Category20')
colors = [color_cycle[i] for i in range(5)]

# Create RGB Map
map = emit_rgb.hvplot.rgb(fontscale=1.5, xlabel='Longitude',ylabel='Latitude',geo=True, frame_width=480, frame_height=480)

# Set up a holoviews points array to enable plotting of the clicked points
xmid = emit_ds.longitude.values[int(len(emit_ds.longitude) / 2)]
ymid = emit_ds.latitude.values[int(len(emit_ds.latitude) / 2)]

first_point = ([xmid], [ymid], [0])
points = hv.Points(first_point, vdims='id')
points_stream = hv.streams.PointDraw(
    data=points.columns(),
    source=points,
    drag=True,
    num_objects=POINT_LIMIT,
    styles={'fill_color': color_cycle.values[0:POINT_LIMIT], 'line_color': 'gray'}
)

clickxy = hv.streams.Tap(source=map, x=147.5, y=-42.7)

# Function to build spectral plot of clicked location to show on hover stream plot
def click_spectra(data):
    coordinates = []
    if data is None or not any(len(d) for d in data.values()):
        coordinates.append(clicked_points[0][0], clicked_points[1][0])
    else:
        coordinates = [c for c in zip(data['x'], data['y'])]
    
    plots = []
    for i, coords in enumerate(coordinates):
        x, y = coords
        data = emit_ds.sel(longitude=x, latitude=y, method="nearest")
        plots.append(
            data.hvplot.line(
                y="reflectance",
                x="wavelengths",
                color=color_cycle,
                label=f"{i}"
            )
        )
        points_stream.data["id"][i] = i
    return hv.Overlay(plots)

# Define the Dynamic Maps
click_dmap = hv.DynamicMap(click_spectra, streams=[points_stream])
#label_dmap = hv.DynamicMap(update_labels, streams=[points_stream])

# Plot the Map and Dynamic Map side by side
hv.Layout(click_dmap + map * points).cols(2).opts(
    hv.opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
    #hv.opts.Labels(xoffset=0.01, yoffset=0.01, bgcolor='gray', text_color='black'),
    hv.opts.Overlay(show_legend=False, show_title=False, fontscale=1.5, frame_height=480)
)

#TODO - This might fit better at a different place.

We can take these selected points and the corresponding reflectance spectra and save them as a `.csv` for later use.

In [ ]:
# data = points_stream.data
# wavelengths = emit_ds.wavelengths.values

# rows = [["id", "x", "y"] + [str(i) for i in wavelengths]]
 
# for p in zip(data['x'], data['y'], data['id']):
#     x, y, i = p
#     spectra = emit_ds.sel(longitude=x, latitude=y, method="nearest").reflectance.values
#     row = [i, x, y] + list(spectra)
#     rows.append(row)

In [ ]:
# with open('data.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerows(rows)

## 2.1 Quality Masking

The EMIT L2A Mask file contains some bands that are direct masks (Cloud, Dilated, Currus, Water, Spacecraft), and some (AOD550 and H2O (g cm-2)) that contain information calculated during the L2A reflectance retrieval. These may be used as additional screening, depending on the application.  The Aggregate Flag is the mask used during EMIT L2B Mineralogy calculations, which we will also use here, but not all users might want this particular mask.

> Note: It is more memory efficient to apply the mask before orthorectifying, so during the automation section we will do that.

In [ ]:
emit_mask = emit_xarray(emit_qa_fp, ortho=True)
emit_mask

List the quality flags contained in the `mask_bands` dimension.

In [ ]:
emit_mask.mask_bands.data.tolist()

As mentioned, we will use the `Aggregate Flag`. Select that band with the `sel` function as we did for wavelengths before.

In [ ]:
emit_aggregate_mask = emit_mask.sel(mask_bands='Aggregate Flag')

Now we can visualize our aggregate quality mask. You may have noticed before that we added a lot of parameters to our plotting function. If we want to consistently apply the same formatting for multiple plots, we can add those agrguments to a dictionary that we can unpack into `hvplot` functions using `**`.

Create two dictionaries with plotting options.

In [ ]:
size_opts = dict(frame_height=405, frame_width=720, fontscale=2)
map_opts = dict(geo=True,alpha=0.7, xlabel='Longitude',ylabel='Latitude')

In [ ]:
emit_aggregate_mask.hvplot.image(cmap='viridis', tiles='ESRI', **size_opts, **map_opts)

Values of 1 in the mask indicate areas to omit. Apply the mask to our EMIT Data by assigning values where the `mask.data == 1` to `np.nan`

In [ ]:
emit_ds.reflectance.data[emit_aggregate_mask.mask.data == 1] = np.nan

We can confirm our masking worked with a spatial plot.

In [ ]:
emit_layer_filtered_plot = emit_ds.sel(wavelengths=850, method='nearest').hvplot.image(cmap='viridis',tiles='ESRI',**size_opts, **map_opts)
emit_layer_filtered_plot

### 2.2 Cropping EMIT data to a Region of Interest

To crop our dataset to our ROI we first need to open a shapefile of the region. Open the included `geojson` for Sedgwick Reserve and Plot it onto our EMIT 850nm reflectance spatial plot. Note that here we don't use use tiles -- there seems to be a bug preventing this.

In [ ]:
shape = gp.read_file("../data/dangermond_boundary.geojson")
shape

In [ ]:
emit_ds.sel(wavelengths=850, method='nearest').hvplot.image(cmap='viridis',**size_opts,**map_opts)*shape.hvplot(geo=True,color='#d95f02',alpha=0.5)

Now use the `clip` function from `rasterio` to crop the data to our ROI using our shape's `geometry` and `crs`. The `all_touched=True` argument will ensure all pixels touched by our polygon will be included.

In [ ]:
emit_cropped = emit_ds.rio.clip(shape.geometry.values,shape.crs, all_touched=True)

Plot the cropped data.

In [ ]:
emit_cropped.sel(wavelengths=850,method='nearest').hvplot.image(cmap='viridis', tiles='ESRI', **size_opts, **map_opts)

### 2.3 Write an output

Lastly for our EMIT dataset, we can write a smaller output that we can use in later notebooks, to calculate Canopy water content or other applications. We use the `granule_id` from the dataset to keep a similar naming convention.

In [ ]:
# Write Clipped Output
#emit_cropped.to_netcdf(f'../data/{emit_cropped.granule_id}_dangermond.nc')

## 3.0 Working with ECOSTRESS L2T Land Surface Temperature and Emissivity

For this example we're only taking a look at the Land Surface Temperature. 

Open the LST file using `open_rasterio` from the `rioxarray` library. Since the file consists of only 1 layer, we can `squeeze` it, removing the `band` dimension.

In [ ]:
eco_lst_ds = rxr.open_rasterio(eco_fp).squeeze('band', drop=True)
eco_lst_ds

#TODO - The ecostress plot is being reprojected automatically - need to write up some explanation text

In [ ]:
eco_lst_ds.hvplot.image(x='x',y='y',**size_opts, cmap='viridis',tiles='ESRI', xlabel='Longitude',ylabel='Latitude',title='ECOSTRESS LST (K)')

We do not need to mask ECOSTRESS, because cloud masking has already been done for the tiled LST product.

In [ ]:
eco_lst_ds_regrid = eco_lst_ds.rio.reproject_match(emit_cropped)

Regridding to the cropped EMIT data automatically restricts us to its spatial ref/bounding box.

In [ ]:
eco_lst_ds_regrid.hvplot.image(geo=True,cmap='inferno',**size_opts)

In [ ]:
eco_dangermond = eco_lst_ds_regrid.rio.clip(shape.geometry.values,shape.crs, all_touched=True)

In [ ]:
eco_dangermond.hvplot.image(geo=True,cmap='inferno',**size_opts, tiles='ESRI',alpha=0.7, title='LST (K)', xlabel='Longitude',ylabel='Latitude')

In [ ]:
eco_outname = f"../data/{eco_fp.split('/')[-1].split('.')[0]}_sedgwick.tif"
eco_sedgwick.rio.to_raster(raster_path=eco_outname, driver='COG')